# Notebook batch-1831-land-cover-mapping

## Imports

In [1]:
!pip install rasterio

In [2]:
from PIL import Image
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import shutil
import os
import rasterio
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Approche globale

- Data preparation
- Construction et entrainement d'un premier modèle (incl. identifier un modèle plus puissant potentiellement)
- Finetuner le modèle
- Appliquer à google map (incl. API)
- Construction de l'interface

## Data preparation

Télécharger l'ensemble des données, les consolder et construire le train, val, test set

**Approche :**
- Télécharger les set de données (OpenG & XBD), consolider l'ensemble
- Identifier le nom des images pour les sets (train/val/test)
- Processer les images (ex: np.transpose)
- Calculer les différentes résolutions disponibles et les cas à appliquer (crop/diviser les images) pour faciliter l'entrainement
- Vérifier si certaines données (esp. labels) sont manquantes
- Construire les dossier train / val + test
- Processer la donnée (ex : /255)

### Uploader les images manquantes de XBD

In [ ]:
########### IL VA FALLOIR UPLOADER LES ELEMENTS MANQUANTS ################

# Code qui regarde un csv, cherche le bon fichier dans la BDD XBD, le transforme en numpy array (code Chat utile)
# et le met dans le bon dossier avec le bon nom via la table de correspondance

In [ ]:
# ✅✅✅✅✅

# On pourrait directement passer à l'étape construire les dossiers train/val/test mais je préfère avoir un bon raw data consolidée

#def xbd_paths(path):
    #On charge le fichier de correspondance pour le manipuler et construire les chemins sources & destination
    #df = pd.read_csv(path, header = None, names = ["source", "destination"])
    #df['source'] = df['source'].str.replace(".png",  ".tif")

    #on construit les chemins sources où chercher les images
    #sources = list(df["source"])
    #sources = ["raw_data/XBD/images/" + source for source in sources]

    #on construit les chemins de destinations où iront les images
    #destinations = list(df["destination"])
    #destination_paths = []
    #dossiers = []
    #for destination in destinations:
        #dossier = "_".join(destination.split("_")[:-1])
        #dossiers.append(dossier) # Certains noms de ville sont composés, il faut tout prendre sauf ce qui vient après le dernier _
        #destination_paths.append("raw_data/" + dossier + "/images/" + destination)
    #return sources, destination_paths, dossiers

In [ ]:
### ✅✅✅✅✅ On construit les variables de paths pour copier les fichiers
#sources, destinations, dossiers = xbd_paths("raw_data/xbd_files.csv")

In [ ]:
### ✅✅✅✅✅ On prend chaque fichier, on le convertit en numpy array et on le met dans les bons dossiers

#for source, destination in zip(sources, destinations):
    #shutil.copy(source, destination)

### Exploration de la donnée et listing de fichiers à prendre

In [ ]:
#✅✅✅✅✅ Création d'un listes avec les noms des fichiers à prendre

#On construit une fonction qui va retourner le chemin vers 1)
#les images du X 2) les images du y - en fonction du paramètre donnee qui est "train", "test", "val"
'''def read_txt(donnee):
    with open(f"raw_data/{donnee}.txt", 'r') as file:
        lines = file.readlines()
        X = []
        y = []
        for line in lines:
            region = line.split("_")[0]
            title = line.replace('\n', '')
            X.append("raw_data/"+ region + "/images/" + title)
            y.append("raw_data/"+ region + "/labels/" + title)
        return X, y'''

'def read_txt(donnee):\n    with open(f"raw_data/{donnee}.txt", \'r\') as file:\n        lines = file.readlines()\n        X = []\n        y = []\n        for line in lines:\n            region = line.split("_")[0]\n            title = line.replace(\'\n\', \'\')\n            X.append("raw_data/"+ region + "/images/" + title)\n            y.append("raw_data/"+ region + "/labels/" + title)\n        return X, y'

In [ ]:
# ✅✅✅✅✅ On crée les listes pour les différents train et test
#X_train_set, y_train_set = read_txt("train")
#X_val_set, y_val_set = read_txt("val")
#X_test_set, y_test_set = read_txt("test")

In [ ]:
# ✅✅✅✅✅ On definit nos set de données finaux (quand X et y existent)
"""X_train_set_final = []
y_train_set_final = []
train_files = []

X_val_set_final = []
y_val_set_final = []
val_files = []

X_test_set_final = []
y_test_set_final = []
test_files = []


for X, y in zip(X_train_set, y_train_set):
    if os.path.isfile(X) == True and os.path.isfile(y) == True:
        X_train_set_final.append(X)
        y_train_set_final.append(y)
        train_files.append(X.split("/")[-1])

for X , y  in zip(X_test_set, y_test_set):
    if os.path.isfile(X) == True and os.path.isfile(y) == True:
        X_test_set_final.append(X)
        y_test_set_final.append(y)
        test_files.append(X.split("/")[-1])


for X, y  in zip(X_val_set, y_val_set):
    if os.path.isfile(X) == True and os.path.isfile(y) == True:
        X_val_set_final.append(X)
        y_val_set_final.append(y)
        val_files.append(X.split("/")[-1])"""

'X_train_set_final = []\ny_train_set_final = []\ntrain_files = []\n\nX_val_set_final = []\ny_val_set_final = []\nval_files = []\n\nX_test_set_final = []\ny_test_set_final = []\ntest_files = []\n\n\nfor X, y in zip(X_train_set, y_train_set): \n    if os.path.isfile(X) == True and os.path.isfile(y) == True:\n        X_train_set_final.append(X)\n        y_train_set_final.append(y)\n        train_files.append(X.split("/")[-1])\n\nfor X , y  in zip(X_test_set, y_test_set): \n    if os.path.isfile(X) == True and os.path.isfile(y) == True:\n        X_test_set_final.append(X)\n        y_test_set_final.append(y)\n        test_files.append(X.split("/")[-1])\n\n\nfor X, y  in zip(X_val_set, y_val_set): \n    if os.path.isfile(X) == True and os.path.isfile(y) == True:\n        X_val_set_final.append(X)\n        y_val_set_final.append(y)\n        val_files.append(X.split("/")[-1])'

### Construction des dossiers train/val/test

In [ ]:
# ✅✅✅✅✅ On construit une fonction qui va aller chercher les images dont le chemin est
#indiqué dans le set et les sauvegarder dans un dossier train ou val (en fonction du X et du y)
#i = 0
# on copie les fichiers pour le train
"""for X_train, y_train in zip(X_train_set_final, y_train_set_final):
    dst_X_train = "clean_data/train/images/"
    os.makedirs(os.path.dirname(dst_X_train), exist_ok=True)
    shutil.copy(X_train, dst_X_train + X_train.split("/")[-1])

    dst_y_train = "clean_data/train/labels/"
    os.makedirs(os.path.dirname(dst_y_train), exist_ok=True)
    shutil.copy(y_train, dst_y_train + y_train.split("/")[-1])"""


#on copie les fichiers sur le val

"""for  X_val, y_val in zip( X_val_set_final, y_val_set_final):

    dst_X_val = "clean_data/val/images/"
    os.makedirs(os.path.dirname(dst_X_val), exist_ok=True)
    shutil.copy(X_val, dst_X_val + X_val.split("/")[-1])

    dst_y_val = "clean_data/val/labels/"
    os.makedirs(os.path.dirname(dst_y_val), exist_ok=True)
    shutil.copy(y_val, dst_y_val + y_val.split("/")[-1])"""

'for  X_val, y_val in zip( X_val_set_final, y_val_set_final):\n\n    dst_X_val = "clean_data/val/images/"\n    os.makedirs(os.path.dirname(dst_X_val), exist_ok=True)\n    shutil.copy(X_val, dst_X_val + X_val.split("/")[-1])\n\n    dst_y_val = "clean_data/val/labels/"\n    os.makedirs(os.path.dirname(dst_y_val), exist_ok=True)\n    shutil.copy(y_val, dst_y_val + y_val.split("/")[-1])'

In [ ]:
### ✅✅✅✅✅ Fonction pour slicer une image
'''def slicing(image_array, nb_tuiles):
    tuiles_arrays = []
    for tuiles_verticales in range(nb_tuiles):
        for tuiles_horizontales in range(nb_tuiles):
            tuiles_arrays.append(image_array[tuiles_verticales * 256 : 256 * (tuiles_verticales + 1),\
                                 tuiles_horizontales * 256: 256 * (tuiles_horizontales + 1)])
    return tuiles_arrays'''

'def slicing(image_array, nb_tuiles):\n    tuiles_arrays = []\n    for tuiles_verticales in range(nb_tuiles):\n        for tuiles_horizontales in range(nb_tuiles):\n            tuiles_arrays.append(image_array[tuiles_verticales * 256 : 256 * (tuiles_verticales + 1),                                 tuiles_horizontales * 256: 256 * (tuiles_horizontales + 1)])\n    return tuiles_arrays'

In [ ]:
'''with rasterio.open("clean_data/train/images/zanzibar_53.tif") as src:
    image_array = src.read()  # (nb_bands, height, width)
    image_array = np.transpose(image_array, (1, 2, 0))
tuiles_arrays = slicing(image_array, 4)'''

'with rasterio.open("clean_data/train/images/zanzibar_53.tif") as src:\n    image_array = src.read()  # (nb_bands, height, width)\n    image_array = np.transpose(image_array, (1, 2, 0))\ntuiles_arrays = slicing(image_array, 4)'

In [ ]:
'''def save_tuile(tuiles_arrays, train_file, data_set, images_or_label ):
    path = "processed_data/" + data_set + "/" + images_or_label + "/"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    for tuile_id, tuile_array in enumerate(tuiles_arrays):
        if tuile_array.shape[-1] == 1 :
            tuile_array = np.squeeze(tuile_array)
        tuile_array = tuile_array.astype("uint8")
        image = Image.fromarray(tuile_array)
        image.save(path + "/"  + train_file.split(".")[0] + "_" + str(tuile_id + 1) + ".tif")'''


'def save_tuile(tuiles_arrays, train_file, data_set, images_or_label ):\n    path = "processed_data/" + data_set + "/" + images_or_label + "/"\n    os.makedirs(os.path.dirname(path), exist_ok=True)\n    for tuile_id, tuile_array in enumerate(tuiles_arrays):\n        if tuile_array.shape[-1] == 1 :\n            tuile_array = np.squeeze(tuile_array)\n        tuile_array = tuile_array.astype("uint8")\n        image = Image.fromarray(tuile_array)\n        image.save(path + "/"  + train_file.split(".")[0] + "_" + str(tuile_id + 1) + ".tif")'

In [ ]:
'''def crop_tuile(data_set):
    if data_set == "train":
        files = train_files
    elif data_set == "val":
        files = val_files
    path = "clean_data/" + data_set + "/"
    #On ouvre chaque image X puis chaque image y. Commençons par les X
    ## Ouverture des X - théoriquement les images seront de même taille
    for file in files:

        with rasterio.open(path + "images/" + file) as src:
            image_array = src.read()  # (nb_bands, height, width)
            image_array = np.transpose(image_array, (1, 2, 0))

        #ensuite on regarde en combien de tuiles on peut le faire
        nb_tuiles = (image_array.shape[0] // 256) # Sur une longueur ou hauteur on peut fiter nb_tuiles + un reste de pixels qui sera perdu
        tuiles_totales = nb_tuiles **2 # ça nous aidera pour le nommage des tuiles (ex = aachen_1_2 - tuile 2 de la photo aachen_1
        #il faudra donc slicer en nb_tuiles à la verticale et à la horizontale

        #On va slicer le X
        tuiles_arrays = slicing(image_array, nb_tuiles)
        #on va sauvegarder les tuiles dans le fichier processed_data/train/images/X
        save_tuile(tuiles_arrays, file, data_set, "images")

        with rasterio.open(path + "labels/" + file) as src:
            image_array = src.read()  # (nb_bands, height, width)
            image_array = np.transpose(image_array, (1, 2, 0))

        #on slide le y de la même manière que le X avec la même convention de nommage (donc le même nom

        tuiles_arrays = slicing(image_array, nb_tuiles)
        #on va sauvegarder les tuiles dans le fichier processed_data/train/images/X
        save_tuile(tuiles_arrays, file, data_set, "labels")'''


'def crop_tuile(data_set):\n    if data_set == "train":\n        files = train_files\n    elif data_set == "val":\n        files = val_files\n    path = "clean_data/" + data_set + "/"\n    #On ouvre chaque image X puis chaque image y. Commençons par les X\n    ## Ouverture des X - théoriquement les images seront de même taille\n    for file in files:\n        \n        with rasterio.open(path + "images/" + file) as src:\n            image_array = src.read()  # (nb_bands, height, width)\n            image_array = np.transpose(image_array, (1, 2, 0))\n        \n        #ensuite on regarde en combien de tuiles on peut le faire\n        nb_tuiles = (image_array.shape[0] // 256) # Sur une longueur ou hauteur on peut fiter nb_tuiles + un reste de pixels qui sera perdu\n        tuiles_totales = nb_tuiles **2 # ça nous aidera pour le nommage des tuiles (ex = aachen_1_2 - tuile 2 de la photo aachen_1\n        #il faudra donc slicer en nb_tuiles à la verticale et à la horizontale\n\n        #On 

In [ ]:
#crop_tuile("train")
#crop_tuile("val")

## Construction du modèle

### Definition du modèle UNET et initialisation

In [78]:
# Weighted crossentropy loss

def weighted_categorical_crossentropy(weights):
    def loss(y_true, y_pred):
        y_pred = tf.keras.backend.clip(y_pred, 1e-7, 1 - 1e-7)
        return tf.reduce_mean(-tf.reduce_sum(weights * y_true * tf.math.log(y_pred), axis=-1))
    return loss

# Exemple d'utilisation :
class_weights = tf.constant([1.0, 2.0, 0.5, 1.5])  # Exemple de poids pour chaque classe
loss_fn = weighted_categorical_crossentropy(class_weights)

In [79]:
# Dice loss

def dice_loss(y_true, y_pred, smooth=1e-6):
    y_pred = tf.keras.backend.clip(y_pred, 1e-7, 1 - 1e-7)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice

In [80]:
# Focal loss

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_pred = tf.keras.backend.clip(y_pred, 1e-7, 1 - 1e-7)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_mean(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt))
    return loss

# Exemple d'utilisation :
loss_fn = focal_loss(alpha=0.25, gamma=2.0)

In [81]:
# Combo loss

def combo_loss(alpha=0.5):
    def loss(y_true, y_pred):
        ce_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        d_loss = dice_loss(y_true, y_pred)
        return alpha * ce_loss + (1 - alpha) * d_loss
    return loss

# Exemple d'utilisation :
loss_fn = combo_loss(alpha=0.5)

In [82]:

def combo_loss_new(alpha=0.5, smooth=1e-6):
    """
    Fonction de perte combinée (Dice Loss + Cross-Entropy Loss).

    Args:
        alpha (float): Poids entre Dice Loss (1-alpha) et Cross-Entropy Loss (alpha).
        smooth (float): Terme pour éviter la division par zéro dans Dice Loss.

    Returns:
        Function: Fonction de perte personnalisée.
    """
    def loss(y_true, y_pred):
        """
        Args:
            y_true (tf.Tensor): Étiquettes réelles, [batch_size, height, width] ou one-hot.
            y_pred (tf.Tensor): Prédictions softmax, [batch_size, height, width, num_classes].

        Returns:
            tf.Tensor: Moyenne de la perte sur le batch.
        """
        # Vérifier si les étiquettes sont one-hot encodées, sinon les encoder
        if len(y_true.shape) == len(y_pred.shape) - 1:  # Si pas one-hot encodé
            num_classes = tf.shape(y_pred)[-1]
            y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

        # Vérifier que y_true et y_pred ont les mêmes dimensions
        assert y_true.shape == y_pred.shape, f"Shapes incompatibles: {y_true.shape} vs {y_pred.shape}"

        # Limiter les prédictions pour éviter les instabilités numériques
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

        # Calcul de Dice Loss
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
        union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
        dice_loss = 1 - (2. * intersection + smooth) / (union + smooth)

        # Calcul de Cross-Entropy Loss
        cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1))

        # Combinaison des deux pertes
        return alpha * cross_entropy_loss + (1 - alpha) * dice_loss

    return loss

In [83]:
def iou_loss(smooth=1e-6):
    """
    Implements the IoU Loss function for image segmentation.

    Args:
        smooth (float): Small value to avoid division by zero.

    Returns:
        Function: IoU loss function.
    """
    def loss(y_true, y_pred):
        """
        Args:
            y_true (tf.Tensor): Ground truth mask, shape [batch_size, height, width, num_classes].
                                Can be one-hot encoded or scalar.
            y_pred (tf.Tensor): Predicted mask, shape [batch_size, height, width, num_classes].
                                Softmax probabilities.

        Returns:
            tf.Tensor: Mean IoU loss over the batch.
        """
        # Ensure y_true is one-hot encoded
        if len(y_true.shape) == len(y_pred.shape) - 1:
            num_classes = tf.shape(y_pred)[-1]
            y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

        # Clip predictions to avoid numerical instability
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

        # Calculate intersection and union
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
        union = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3]) - intersection

        # IoU calculation
        iou = (intersection + smooth) / (union + smooth)

        # IoU loss
        iou_loss = 1 - iou

        # Mean IoU loss across the batch
        return tf.reduce_mean(iou_loss)

    return loss

In [96]:
def combined_loss(alpha=0.5):
    """
    Combines IoU Loss and Dice Loss.

    Args:
        alpha (float): Weighting between IoU Loss and Dice Loss.

    Returns:
        Function: Combined loss function.
    """
    iou_loss_fn = iou_loss()

    def loss(y_true, y_pred):
        return alpha * iou_loss_fn(y_true, y_pred) + (1 - alpha) * dice_loss(y_true, y_pred)

    return loss

In [99]:
def unet(input_shape, num_classes):
    inputs = Input(input_shape)
    # Encoder Block 1
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    # Encoder Block 2
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    # Encoder Block 3
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    # Encoder Block 4
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    # Bridge
    bridge = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    bridge = Conv2D(1024, (3, 3), activation='relu', padding='same')(bridge)
    # Decoder Block 1
    up1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bridge)
    concat1 = Concatenate()([up1, conv4])
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(concat1)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    # Decoder Block 2
    up2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)
    concat2 = Concatenate()([up2, conv3])
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat2)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    # Decoder Block 3
    up3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
    concat3 = Concatenate()([up3, conv2])
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    # Decoder Block 4
    up4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
    concat4 = Concatenate()([up4, conv1])
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat4)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    # Output Layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv8)
    # Model
    model = Model(inputs, outputs, name="U-Net")
    return model
model = unet((256, 256, 3),9)
model.summary()
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18            │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_342 (Conv2D)       │ (None, 256, 256, 64)   │          1,792 │ input_layer_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_343 (Conv2D)       │ (None, 256, 256, 64)   │         36,928 │ conv2d_342[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_72          │ (None, 128, 128, 64)   │              0 │ conv2d_343[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_344 (Conv2D)       │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_72[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_345 (Conv2D)       │ (None, 128, 128, 128)  │        147,584 │ conv2d_344[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_73          │ (None, 64, 64, 128)    │              0 │ conv2d_345[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_346 (Conv2D)       │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_73[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_347 (Conv2D)       │ (None, 64, 64, 256)    │        590,080 │ conv2d_346[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_74          │ (None, 32, 32, 256)    │              0 │ conv2d_347[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_348 (Conv2D)       │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_74[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_349 (Conv2D)       │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_348[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_75          │ (None, 16, 16, 512)    │              0 │ conv2d_349[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_350 (Conv2D)       │ (None, 16, 16, 1024)   │      4,719,616 │ max_pooling2d_75[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_351 (Conv2D)       │ (None, 16, 16, 1024)   │      9,438,208 │ conv2d_350[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_72       │ (None, 32, 32, 512)    │      2,097,664 │ conv2d_351[0][0]       │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_72       

 Total params: 31,032,265 (118.38 MB)

 Trainable params: 31,032,265 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

### Définition des X & y

In [100]:
X_train = []
y_train = []
path_images = "/content/drive/MyDrive/land-cover-mapping/Sandbox data/images/"
for path in os.listdir(path_images)[0:10]:
    with rasterio.open(path_images + path) as src:
        image_array = src.read()  # (nb_bands, height, width)
        image_array = np.transpose(image_array, (1, 2, 0))
        X_train.append(image_array)

path_labels = "/content/drive/MyDrive/land-cover-mapping/Sandbox data/labels/"
for path in os.listdir(path_labels)[0:10]:
    with rasterio.open(path_labels + path) as src:
        image_array = src.read()  # (nb_bands, height, width)
        image_array = np.transpose(image_array, (1, 2, 0))
        y_train.append(image_array)

In [101]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [102]:
y_cat = to_categorical(y_train)

In [103]:
y_cat.shape

(10, 256, 256, 9)

In [104]:
es = EarlyStopping(patience = 10, restore_best_weights = True)

In [105]:
model.fit(X_train, y_cat, validation_split=0.2, epochs=200, batch_size=16, callbacks = [es])

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.0226 - loss: 13.8753 - val_accuracy: 0.2968 - val_loss: 15.5963
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 730ms/step - accuracy: 0.1289 - loss: 19.1472 - val_accuracy: 0.0522 - val_loss: 10.6186
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 621ms/step - accuracy: 0.1790 - loss: 8.2801 - val_accuracy: 0.0269 - val_loss: 8.3352
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step - accuracy: 0.0440 - loss: 7.2227 - val_accuracy: 0.1292 - val_loss: 6.1038
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step - accuracy: 0.1873 - loss: 5.4066 - val_accuracy: 0.2566 - val_loss: 4.6313
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step - accuracy: 0.2262 - loss: 4.5283 - val_accuracy: 0.2038 - val_loss: 4.1712
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step - accuracy: 0.2016 - loss: 4.0759 - val_accuracy: 0.1228 - val_loss: 3.4593
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step - accuracy: 0.1500 - loss: 3.2141 - val_accuracy: 0.0781 

In [ ]:
with rasterio.open("xxx") as src:
    image_array = src.read()  # (nb_bands, height, width)
    image_array = np.transpose(image_array, (1, 2, 0))

RasterioIOError: content/drive/MyDrive/land-cover-mapping/Sandbox data/images/aachen_57_8.tif: No such file or directory

In [ ]:
with rasterio.open("processed_data/val/labels/aachen_57_8.tif") as src:
    image_array_y = src.read()  # (nb_bands, height, width)

In [ ]:
single_band = image_array_y[0]  # Supposons que `nb_bands` = 1

# Afficher avec imshow
plt.imshow(single_band)  # Utiliser cmap='gray' pour les niveaux de gris
plt.colorbar()  # Afficher une barre de couleurs pour visualiser les valeurs
plt.title("Single Band Image")
plt.show()

In [ ]:
image_array_y.shape

In [ ]:
np.expand_dims(image_array, axis = 0).shape

In [ ]:
y_pred = model.predict(np.expand_dims(image_array, axis = 0))

In [ ]:
y_pred.shape

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,15))
ax[0].imshow(np.squeeze(tf.argmax(y_pred, axis=-1, output_type=tf.int32).numpy()))
ax[1].imshow(image_array)

In [ ]:
image_array_y.shape

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,15))
ax[0].imshow(np.squeeze(tf.argmax(y_pred, axis=-1, output_type=tf.int32).numpy()))
ax[1].imshow(image_array)

## Next step model comme VG16 (regarder Notion)